### Importing required libraries

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import keras

### Reading the data

In [ ]:
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.cifar10.load_data()
train_imgs = train_imgs/ 255.0
test_imgs = test_imgs/ 255.0

num_classes = 10
train_lbls = np.squeeze(keras.utils.to_categorical(train_lbls, num_classes))
test_lbls = np.squeeze(keras.utils.to_categorical(test_lbls, num_classes))

### Dispaly of data

In [ ]:
img = (train_imgs[400].squeeze())
plt.imshow(img, cmap='gray')

### Addition of Gaussian Noise to the dataset and display of the noisy image

In [6]:
def add_gaussian_noise(images, std_dev):
    n_imgs = images.shape[0]
    n_rows = images.shape[1]
    n_cols = images.shape[2]
    n_chan = images.shape[3]
    noise = np.random.normal(0.0 ,std_dev , (n_imgs, n_rows, n_cols, n_chan))
    noisy_X = images + noise
    noise_images = np.clip(noisy_X, 0., 1.)
    return noise_images

noise_test_imgs = add_gaussian_noise(test_imgs, 1)  #Depending on the level of noise, value is assigned

noise_img = (noise_train_imgs[15].squeeze())
plt.imshow(noise_img, cmap="gray")

### Creating model 'BF'

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape= (32,32,3)),
    tf.keras.layers.Conv2D(64,3, strides = (1, 1), padding='same', use_bias=True,activation = 'relu'),
    tf.keras.layers.MaxPool2D((2, 2),(2, 2)),
    tf.keras.layers.Conv2D(64, 3, strides = (1, 1), padding='same', use_bias=True,activation = 'relu'),
    tf.keras.layers.GlobalMaxPooling2D(data_format='channels_last'),
    tf.keras.layers.Dense(10, activation = 'sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
])
model.summary()

### Initializing parameters for training

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import callbacks

def step_decay(epoch):
    initial_lr=0.001
    decay_rate = 0.01
    decay_step = 30
    lrate = initial_lr * decay_rate ** (epoch/decay_step)
    print("learning_rate")
    print(lrate)
    return lrate

Adam = tf.keras.optimizers.Adam(lr=0.0)
model.compile(loss='categorical_crossentropy', optimizer = Adam, metrics=['accuracy'])

lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

### Compiling the model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = Adam, metrics=['accuracy'])

### Fit the model

In [ ]:
#With Noise
history_dict = dict()
for i in range(1, 101):
    noise_train_imgs = add_gaussian_noise(train_imgs, 1)  #noise is either 1 or 2, depending on the noise level
    history_dict['epoch_%i' % i] = model.fit(noise_train_imgs, train_lbls, epochs=1, batch_size=128, validation_split=0.1, callbacks = callbacks_list)

In [ ]:
print(history.history.keys())

### Display learning curve

In [ ]:
# With noise

train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

for i in range(1,101):
    print(i)
    val_loss = history_dict['epoch_%i' % i].history['val_loss']
    val_acc = history_dict['epoch_%i' % i].history['val_accuracy']
    acc = history_dict['epoch_%i' % i].history['accuracy']
    loss = history_dict['epoch_%i' % i].history['loss']

    train_loss_list.append(loss)
    train_acc_list.append(acc)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

### Test the model

In [ ]:
#With Noise
model.evaluate(noise_test_imgs, test_lbls)

### Predictions of the model

In [ ]:
predictions = model.predict(test_imgs)

In [ ]:
n = 4
plt.figure(figsize=(15,15))
for i in range(n):
    plt.subplot(1,n,i+1)
    plt.imshow(test_imgs[i])
    plt.title("Lable:{}\nPredicted:{}".format(test_lbls[i],np.argmax(predictions[i])))
    plt.axis='off'
plt.show()

### Saving the model and the predictions

In [ ]:
model.save_weights('Model_BF.h5') #Depending on the level of noise and the model, name is given

In [ ]:
model_predictions = []
ground_truth_values = []
for i in range(len(test_imgs)):
    model_predictions.append(np.argmax(predictions[i]))
      ground_truth_values.append(np.argmax(test_lbls[i]))


correct_prediction = [i if i==j else 0 for i, j in zip(model_predictions, ground_truth_values)]
import pandas as pd
prediction_df = pd.DataFrame(correct_prediction, columns=['result']).to_csv('BF_1_prediction.csv', index=False)